In [2]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from pyvis.network import Network
from tqdm import tqdm
import pickle


In [4]:
# Load in Graph and Artist Data
with open("./saved_objects/main_graph.pkl", "rb") as f:
    G = pickle.load(f)
with open("./saved_objects/artist_info.pkl", "rb") as f:
    artist_info = pickle.load(f)

 # Load previous community detection results   
with open("./saved_objects/community_subgraphs.pkl", "rb") as f:
    comm_subgraphs = pickle.load(f)
with open("./saved_objects/genres_by_community.pkl", "rb") as f:
    genres_by_comm= pickle.load(f)

# 4. Centrality Analysis

## _Global_

In [13]:

degree_centrality = sorted(nx.degree_centrality(G), key=lambda x: x[1], reverse=True)
eigenvector_centrality = sorted(nx.eigenvector_centrality(G), key=lambda x: x[1], reverse=True)
pagerank_centrality = sorted(nx.pagerank(G, alpha=0.85), key=lambda x: x[1], reverse=True)

# Too Slow!
# closeness_centrality = nx.closeness_centrality(G)
# betweenness_centrality = nx.betweenness_centrality(G)

## _Local by subgraph_

In [25]:
# degree_comms = sorted(
#     {idx: nx.degree_centrality(g) for idx, g in enumerate(comm_subgraphs)},
#     key=lambda k, v: np.mean(v, reverse=True),
# )
# closeness_comms = sorted(
#     {idx: nx.closeness_centrality(g) for idx, g in enumerate(comm_subgraphs)},
#     key=lambda k, v: np.mean(v, reverse=True),
# )
betweenness_comms = {idx: nx.betweenness_centrality(g) for idx, g in enumerate(comm_subgraphs)}
eigenvector_comms = {idx: nx.eigenvector_centrality_numpy(g) for idx, g in enumerate(comm_subgraphs)}
katz_comms = {idx: nx.katz_centrality_numpy(g) for idx, g in enumerate(comm_subgraphs)}
# pagerank_comms = sorted(
#     {idx: nx.pagerank(g, alpha=0.85) for idx, g in enumerate(comm_subgraphs)},
#     key=lambda x: np.mean(x[1], reverse=True),
# )

In [49]:
lambda_1 = 1.0
lambda_2 = 1.0
lambda_3 = 1.0
total_centrality = {
    idx: {
        node: lambda_1 * betweenness_comms[idx][node]
        + lambda_2 * eigenvector_comms[idx][node]
        + lambda_3 * katz_comms[idx][node]
        for node in g.nodes()
    }
    for idx, g in enumerate(comm_subgraphs)
}

In [54]:
# Print top artists in a subgraph by compounded centrality
subgraph = 0
top = sorted([k for k in comm_subgraphs[subgraph].nodes()], key=lambda x: total_centrality[subgraph][x], reverse=True)[0:25]
for t in top:
    print(artist_info[t][0])

Wiz Khalifa
Chris Brown
Lil Wayne
Ty Dolla $ign
Gucci Mane
Snoop Dogg
T.I.
French Montana
Future
Tyga
Nas
Lil Durk
T-Pain
Meek Mill
Waka Flocka Flame
Rick Ross
Young Thug
Nelly
Yo Gotti
Big Sean
Pusha T
Kanye West
DJ Khaled
Kehlani
Diplo


In [64]:
def num_to_remove_centrality(G, sorted_nodes):
    G1 = G.copy()
    count = 0
    while nx.number_connected_components(G1) == 1:
        G1.remove_node(sorted_nodes[count])
        count += 1
    return count

for idx, subgraph in enumerate(comm_subgraphs):
    
    num_rem_BC = num_to_remove_centrality(subgraph, sorted([k for k in subgraph.nodes()], key=lambda x: betweenness_comms[idx][x], reverse=True))
    num_rem_EC = num_to_remove_centrality(subgraph, sorted([k for k in subgraph.nodes()], key=lambda x: eigenvector_comms[idx][x], reverse=True))
    num_rem_KC = num_to_remove_centrality(subgraph, sorted([k for k in subgraph.nodes()], key=lambda x: katz_comms[idx][x], reverse=True))

1
1
1
1
1
1
1
1


KeyboardInterrupt: 